[View in Colaboratory](https://colab.research.google.com/github/sbalsky/DressForWeather/blob/master/WeatherBot.ipynb)

In [7]:
'''
Bot to tweet clothing/gear for the weather in Seattle, daily except if clear 
and within 5 degrees of day before.

Steps: 1) Get weather from Open Weather Map API (temp, precipitation, wind);
2) Move existing "today" file to "yesterday" file & save today's high/low temps 
to file; 3) Evaluate weather criteria & set what to wear for each combination; 
4) Tweet 

To do before finished: 1) Add error checking; 2) Learn about how to test 
properly & implement enough of that to demonstrate awareness; 3) Linting

Possible future additions: 1) UV (alert if sunscreen extra needed) & atmospheric
conditions (smoke etc), 2) more finely-grained rain/snow detail, 3) Interactive! 
Tweet location to bot, get back weather for location. 4) Location-aware
'''

from requests import get
# from datetime import datetime, timezone

# ! pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

module_gdrive_id = '1kNqEy6VOyx3ci-V0tItQdtlLSAuyoiKd'
gdrive_io = drive.CreateFile({'id':module_gdrive_id})
gdrive_io.GetContentFile('gdrive_io.py')
import gdrive_io as gd
 

# Step 1: Get today's forecast

APP_KEY = "a43598cd3c1324937c14aa6d6b66f66c"
CITY_ID = "5809844"  # From list of OWM city IDs: http://bulk.openweathermap.org/sample/
DAY = 24 * 60 * 60  # seconds (UNIX time)

def Kelvin_to_F(tempK):
  tempF = round(tempK*9/5 - 459.67, 0)
  return tempF

def WindConvert(meters_per_second):
  mph = 2.23694 * meters_per_second
  return mph

def WindChill(t, mps):
  if mps < 4.8 or t > 50:  # formula not defined outside these parameters
    return t
  else: 
    ws = WindConvert(mps)
    chill = 35.74 + (0.6215 * t) - (35.75 * ws^0.16) + (0.4275 * t * ws^0.16)
    return round(chill, 0)
  
def WeatherToday(): 
  weather_url = "http://api.openweathermap.org/data/2.5/forecast?id=" + CITY_ID + "&APPID=" + APP_KEY
    # 5 day forecast, see https://openweathermap.org/forecast5 for API documentation
  raw = get(weather_url).json()
  current = raw["list"]  
    # strips JSON header info (start at data, which is formatted as a list of dictionaries, some elements of which are also dictionaries)
  today = current[0]["dt"] # first date in list, in UNIX timestamp format

  temp_mins = []
  temp_maxs = []
  winds = []
  weather_types = []

  for i in range(0, 4): # next 15 hours (each data point is 3h)
    temp_mins.append(current[i]["main"]["temp_min"])
    temp_maxs.append(current[i]["main"]["temp_max"])
    winds.append(current[i]["wind"]["speed"])
    
    weather = current[i]["weather"]
    w = weather[0]["main"]
    id = weather[0]["id"]  # Chart of IDs to weather type: https://openweathermap.org/weather-conditions
    if id < 300:  # categorize thunderstorms as rain
      weather_types.append("Rain")
    elif id < 700:
      weather_types.append(w)
    else:  # ignore non-precipitation types >=700
      pass  
    i+=1

  # replace real temps with windchill temps (only care how it feels)
  mins=[]
  maxs=[]
  mph=[]
  for a, b, c in zip(winds, temp_mins, temp_maxs):
    ws = WindConvert(a)
    t_min = Kelvin_to_F(b)
    t_max = Kelvin_to_F(c)
    mins.append(WindChill(t_min, ws))
    maxs.append(WindChill(t_max, ws))
    mph.append(ws)

  temp_low = min(mins)
  temp_high = max(maxs)
  wind = max(mph)

  unique_types = list(set(weather_types))
  if len(unique_types) == 0:
    return {"date": today, "temp_low": temp_low, "temp_high": temp_high, "wind": wind, "type": ""}
  else:
    precip_max = max(unique_types) # Drizzle < Rain < Snow; strongest conveniently in alphabetical order
    return {"date": today, "temp_low": temp_low, "temp_high": temp_high, "wind": wind, "type": precip_max}  

wt = WeatherToday()
today = wt["date"]
# today_utc = datetime.utcfromtimestamp(today)
# yday = today_utc.replace(day=today_utc.day-1)


# Step 2: Move previous day's temps to yesterday file, save today's to today file

def Content(data_list):
  text = ",".join('{}'.format(val) for val in data_list)
  return text
  
today_file = "today.txt"
yday_file = "yesterday.txt"

def Yesterday(day): 
  # only rewrite files if it's actually a new day
  t_id = gd.GetID(today_file)
  y_id = gd.GetID(yday_file)
  
  if t_id == None: # no "today" file created yday; make up values for yday
    # print("File {} did not exist; invented values for yday".format(today_file))
    date, low, high = day - DAY, 50, 70
    y_list= [date, low, high]
  else: # get yday values from today file that was written yesterday
    y_txt = gd.OpenFile(t_id)
    y_list = y_txt.split(",")
    date, low, high = y_txt.split(",")

  if int(date) < day:
    gd.Write(yday_file, Content(y_list))
    t_list = [wt["date"], wt["temp_low"], wt["temp_high"]]
    gd.Write(today_file, Content(t_list))
  else:
    if y_id == None: # no "yesterday" file, make up values
      date, low, high = day - DAY, 55, 75
      y_list = [date, low, high]
    else:
      y_txt = gd.OpenFile(y_id)
      y_list = y_txt.split(",")
      date, low, high = y_txt.split(",")
      gd.Write(yday_file, Content(y_list))

  return {"yday_low": float(low), "yday_high": float(high)}

wy = Yesterday(today)

# Step 3: Evaluate relevant conditions & report

SHORT = 80
LONG = 70
COAT = 50
HAT = 40
ALL = 20
BREEZE = 8
WIND = 15

layers = [SHORT, LONG, COAT, HAT, ALL]
layer_desc = ["Short sleeves today! ", "Wear long sleeves. ", "You'll want a jacket. ", "Time to break out the winter coat, & remember a hat/gloves. ", "Bundle up!!! "]
layer_remove = ["No need for long sleeves today. ","Just a sweater is fine. ","You can drop down to a less puffy jacket now. ","OK with just the standard winter stuff. "]
layer_abbrev = ["T-shirt. ", "Sweater. ", "Jacket. ", "Coat/hat/gloves. ", "Bundle up! "]

yday_low = wy["yday_low"]
yday_high = wy["yday_high"]
today_low = wt["temp_low"]
today_high = wt["temp_high"]
precip = wt["type"]
wind_speed = wt["wind"]

abbrev = 0

# test replace values
# today_low = 10
# today_low = 90
# today_high = 40
# today_high = 90
# yday_low = 20
# yday_low = 80
# yday_high = 20
# yday_high = 80
# precip = "Snow"
# precip = "Rain"
# precip = "Drizzle"
# wind_speed = 10
# wind_speed = 20
# long_twt = "x" * 300

def Short(t):
  if t >= SHORT:
    short = 1
  else:
    short = 0
  return short

def LayerType(t, layers):
  i = 0
  layer_type = 0
  while t <= layers[i]:
    layer_type = i
    i+=1
    if i == len(layers):
      break
  return layer_type

def InclDiff(d, y):
  diff = int(round(d - y, 0))
  if diff >= 5:
    add = "Low is {} degrees warmer than yesterday. ".format(diff)
  elif diff <= -5:
    add = "Low is {} degrees cooler than yesterday. ".format(-diff)
  else:
    add = ""
  return add

def Shoes(precip_type, temp):
  if precip_type == "":
    shoes = ""
    shoes_abbr = ""
  elif precip_type == "Snow" or temp <= HAT:
    shoes = "Warm dry boots with wool socks are in order. "
    shoes_abbr = "Warm boots. "
  elif precip_type == "Rain":
    shoes = "Waterproof boots would be a good idea. "
    shoes_abbr = "Waterproof shoes. "
  else:  # drizzle
    shoes = "Not the day for suede shoes. "
    shoes_abbr = ""

  if abbrev == 1:
    return shoes_abbr
  else:
    return shoes

def Outerwear(precip, wind, layer):
  if wind >= WIND and precip == "" and layer == 0:
    outer = "Consider a light button-down: {} mph wind. ".format(round(wind_speed, 0))
    outer_abbr = "Button-down for wind. "
  elif (wind >= WIND and 1 <= layer <= 2) or (wind >= BREEZE and layer == 2):
    if precip == "" or precip == "Drizzle":
      outer = "Bring a windbreaker and earband. "
      outer_abbr = "Windbreaker/earband. "
    else:
      outer = "Bring a water-resistant windbreaker and hat/umbrella. "
      outer_abbr = "Windbreaker/hat. "
  else:
    outer = 1
    outer_abbr = 1
  
  if abbrev == 1:
    return outer_abbr
  else:
    return outer

def TweetTooLong(twt):
  MAX_LENGTH = 280
  if len(twt) <= MAX_LENGTH:
    return False
  else:
    return True

  
yday_short = Short(yday_high)
today_short = Short(today_high)

if today_short > yday_short:
  display = layer_desc[0]
else:
  display = ""

display += str(InclDiff(today_low, yday_low))
  
yday_layer = LayerType(yday_low, layers)
today_layer = LayerType(today_low, layers)

outerwear = Outerwear(precip, wind_speed, today_layer)
shoes = Shoes(precip, today_low)

if outerwear == 1:
  if today_layer > yday_layer:
    outerwear = layer_desc[today_layer]
  elif today_layer < yday_layer:
    outerwear = layer_remove[today_layer]
  else:
    outerwear = ""
else:
  pass
  
tweet_string = display + outerwear + shoes
# tweet_string = long_twt

if TweetTooLong(tweet_string):
  abbrev = 1
  shoes = Shoes(precip, today_low)
  outerwear = Outerwear(precip, wind_speed, today_layer)
  if outerwear == 1 and today_layer != yday_layer:
    outerwear = layer_abbrev[today_layer]
  else:
    outerwear = ""
    
  tweet_string = outerwear + shoes
else:
  pass

weather_text = ", ".join(["{} = {}".format(key, val) for key, val in wt.items()])
print("\nBased on today's weather: "+", ".join(["{} = {}".format(key, val) for key, val in wt.items()]))
print("& yday's high of {}, low of {}".format(wy["yday_high"],wy["yday_low"]))
if len(tweet_string)>0:
  print("\nTweet: " + tweet_string)
else: 
  print("\nNo tweet: Clear & similar to yesterday")



Based on today's weather: date = 1534993200, temp_low = 58.0, temp_high = 76.0, wind = 8.6569578, type = Rain
& yday's high of 70.0, low of 50.0

Tweet: Low is 8 degrees warmer than yesterday. Just a sweater is fine. Waterproof boots would be a good idea. 
